In [36]:
from torch.nn.utils.rnn import pad_sequence
from Bert import Bert
import torch
from tokenizers.implementations import BertWordPieceTokenizer

In [37]:
_embedding_dim = 384
_hidden_size = 3072
_num_head = 12
_num_layers = 12
_tokenizer = BertWordPieceTokenizer("custom/vocab.txt")

In [38]:
bert = Bert(_embedding_dim, _hidden_size, _num_head, 128, _num_layers, _tokenizer, device='cuda:0')
classifier = torch.nn.Linear(_embedding_dim, 2, device='cuda:0')
bert.load_state_dict(torch.load("down_stream_bert.pth"))
classifier.load_state_dict(torch.load("down_stream.pth"))
bert.eval()
classifier.eval()

Linear(in_features=384, out_features=2, bias=True)

1. 把句子转换成张量

In [95]:
sentences = ['Wow, this product is so perfect!', 'My dog was dead yesterday', 'This is not bad, it is fine',
             'After using this thing, My dog was dead yesterday']
ids = pad_sequence([torch.tensor(_tokenizer.encode(i).ids, dtype=torch.long, device='cuda:0') for i in sentences], batch_first=True)
ids

tensor([[    2, 28479,    16,  1829,  4194,  1692,  2101,  7671,     5,     3,
             0,     0],
        [    2,  2790,  5424,  1683,  4617, 27553,     3,     0,     0,     0,
             0,     0],
        [    2,  1829,  1692,  1831,  4226,    16,  1745,  1692,  5965,     3,
             0,     0],
        [    2,  1878,  3105,  1829,  7976,    16,  2790,  5424,  1683,  4617,
         27553,     3]], device='cuda:0')

2. 获取输入的第一个分类token的值

In [96]:
with torch.no_grad():
    score = bert.forward(ids, torch.not_equal(ids, 0))
    score = classifier(score)[:, 0, :]
score

tensor([[-0.6545,  4.2176],
        [ 1.0532,  1.0461],
        [-0.7777,  3.9766],
        [ 2.3728, -0.4527]], device='cuda:0')

3. 获取概率

In [97]:
possibility = torch.nn.Softmax(dim=-1)(score)
possibility

tensor([[0.0076, 0.9924],
        [0.5018, 0.4982],
        [0.0085, 0.9915],
        [0.9440, 0.0560]], device='cuda:0')

4. 获取最高概率(0.9924)

In [98]:
torch.argmax(possibility, dim=-1)

tensor([1, 0, 1, 0], device='cuda:0')